In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv(r"C:\Users\jorda\OneDrive\Documents\FloodDamagePrediction\Fema.csv.csv", low_memory = False)

In [3]:
# Calculate the proportion of observations with waterDepth <= n (1,2,3,4,5,6) per date and censusBlockGroupFips
df['proportionLeqOne'] = df.groupby(['dateOfLoss', 'censusBlockGroupFips'])['waterDepth'].transform(
    lambda x: (x <= 1).mean()
)

df['proportionLeqTwo'] = df.groupby(['dateOfLoss', 'censusBlockGroupFips'])['waterDepth'].transform(
    lambda x: (x <= 2).mean()
)

df['proportionLeqThree'] = df.groupby(['dateOfLoss', 'censusBlockGroupFips'])['waterDepth'].transform(
    lambda x: (x <= 3).mean()
)

df['proportionLeqFour'] = df.groupby(['dateOfLoss', 'censusBlockGroupFips'])['waterDepth'].transform(
    lambda x: (x <= 4).mean()
)

df['proportionLeqFive'] = df.groupby(['dateOfLoss', 'censusBlockGroupFips'])['waterDepth'].transform(
    lambda x: (x <= 5).mean()
)

df['proportionLeqSix'] = df.groupby(['dateOfLoss', 'censusBlockGroupFips'])['waterDepth'].transform(
    lambda x: (x <= 6).mean()
)

In [4]:
df['proportionLeqOne'].head()

0    1.000000
1    0.928571
2    0.888889
3    1.000000
4    1.000000
Name: proportionLeqOne, dtype: float64

In [ ]:
# Ensure that dateOfLoss is of datetime type
df['dateOfLoss'] = pd.to_datetime(df['dateOfLoss'])

# Define a function that, given a group dataframe and a date, 
# returns the median, 25th, 75th, and 90th percentile of waterDepth in the 3-day window centered on that date
def get_stats_in_window(group_df, date):
    mask = (group_df['dateOfLoss'] >= date - pd.Timedelta(days=1)) & (group_df['dateOfLoss'] <= date + pd.Timedelta(days=1))
    selected_water_depths = group_df.loc[mask, 'waterDepth']
    return {
        'medWaterDepth': selected_water_depths.median(),
        'p25WaterDepth': selected_water_depths.quantile(0.25),
        'p75WaterDepth': selected_water_depths.quantile(0.75),
        'p90WaterDepth': selected_water_depths.quantile(0.90)
    }

# Iterate over all groups and all dates, and calculate the statistics in the 3-day window
for fips, group_df in df.groupby('censusBlockGroupFips'):
    for date in group_df['dateOfLoss'].unique():  # using unique to avoid recalculating for same date
        stat = get_stats_in_window(group_df, date)
        mask = (df['censusBlockGroupFips'] == fips) & (df['dateOfLoss'] == date)
        df.loc[mask, 'medWaterDepth'] = stat['medWaterDepth']
        df.loc[mask, 'p25WaterDepth'] = stat['p25WaterDepth']
        df.loc[mask, 'p75WaterDepth'] = stat['p75WaterDepth']
        df.loc[mask, 'p90WaterDepth'] = stat['p90WaterDepth']



In [ ]:
#get new variable of difference between observed water depth and median and 90th quantile per 3 day window water depth grouped by census block

df['diffMedWaterDepth'] = df['waterDepth'] - df['medWaterDepth']
df['diff90WaterDepth'] = df['waterDepth'] - df['p90WaterDepth']
